In [6]:
import pandas as pd
from openai import OpenAI
import random
import os
import re

In [844]:
category = "mecaz"
idiom = "Üstüne almak"
model ="gpt-4-turbo"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OpenAI Key",
)

In [845]:
if not os.path.exists('response_data.csv'):
    pd.DataFrame(columns=["raw_response","category", "idiom", "type"]).to_csv('response_data.csv', index=False)


In [846]:
df_structured = pd.DataFrame(columns=["submission", "category", "idiom", "type"])

**Zero-Shot**

In [852]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. Lütfen cümlelerinizde benzer yapılar kullanmaktan kaçının ve deyimleri doğru ve {category} anlamıyla kullanarak cümleler oluşturun. İnsan isimlerini kullanarak cümle kurmamalısın !"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [853]:
chat_session = ChatSession(category=category)

Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. Lütfen cümlelerinizde benzer yapılar kullanmaktan kaçının ve deyimleri doğru ve mecaz anlamıyla kullanarak cümleler oluşturun. İnsan isimlerini kullanarak cümle kurmamalısın !


In [882]:
type = "Zero-shot"
step = 8

In [883]:
prompt = f"""
"{idiom}", Türkçe bir deyimidir. Bu deyimi hem gerçek hem de mecaz anlamda kullanabiliriz. Lütfen bu deyimi {category} anlamda kullanarak örnek 5 tane cümle yaz. Yanıt olarak sadace cümleleri dön.
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


"Üstüne almak", Türkçe bir deyimidir. Bu deyimi hem gerçek hem de mecaz anlamda kullanabiliriz. Lütfen bu deyimi mecaz anlamda kullanarak örnek 5 tane cümle yaz. Yanıt olarak sadace cümleleri dön.





In [884]:
responses

['1. Müdürün genel uyarısının ardından, hepsi kafalarını öne eğdi ama sadece biri sözleri üstüne alarak hemen aksiyona geçti.\n2. Projede yaşanan aksaklıkla ilgili kimse konuşmak istemezken, o, hataları üstüne alıp çözüm bulmaya çalıştı.\n3. Eleştiriler genellikle genel olarak söylenir ama bazen biri çıkar bu sözleri üstüne alır ve işleri rayına oturtur.\n4. Arkadaş gruplarında sıkça karşılaşılan sorunlarda, herkes sessiz kalabilir ama cesur yürekler bu sorunları üstüne alarak ortamı yumuşatır.\n5. Herkes projedeki başarısızlıkla ilgili endişeliyken, proje yöneticisi cesaretle tüm suçu üstüne alarak ekip üyelerini korudu.',
 '1. Grup içinde yapılan uyarıları kimse ciddiye almazken, o, söylenenleri üstüne alarak hemen işe koyuldu.\n2. Toplantıda dile getirilen eksiklikler üzerine konuşulduğunda, o her zamanki gibi eleştiriyi üstüne alıp düzeltmek için adım attı.\n3. Kimse kabul etmek istemezken, sorunun farkında olan tek kişi olarak o, hatayı üstüne alarak özür diledi.\n4. Her ne kadar 

In [887]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [888]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,"Müdürün genel uyarısının ardından, hepsi kafal...",mecaz,Üstüne almak,Zero-shot
1,Projede yaşanan aksaklıkla ilgili kimse konuşm...,mecaz,Üstüne almak,Zero-shot
2,Eleştiriler genellikle genel olarak söylenir a...,mecaz,Üstüne almak,Zero-shot
3,Arkadaş gruplarında sıkça karşılaşılan sorunla...,mecaz,Üstüne almak,Zero-shot
4,Herkes projedeki başarısızlıkla ilgili endişel...,mecaz,Üstüne almak,Zero-shot
5,Grup içinde yapılan uyarıları kimse ciddiye al...,mecaz,Üstüne almak,Zero-shot
6,Toplantıda dile getirilen eksiklikler üzerine ...,mecaz,Üstüne almak,Zero-shot
7,"Kimse kabul etmek istemezken, sorunun farkında...",mecaz,Üstüne almak,Zero-shot
8,Her ne kadar kimsenin hatası olarak görünmese ...,mecaz,Üstüne almak,Zero-shot
9,Ekip üyeleri genellikle hataları kabul etmekte...,mecaz,Üstüne almak,Zero-shot


In [889]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,"Müdürün genel uyarısının ardından, hepsi kafal...",mecaz,Üstüne almak,Zero-shot
1,Projede yaşanan aksaklıkla ilgili kimse konuşm...,mecaz,Üstüne almak,Zero-shot
2,Eleştiriler genellikle genel olarak söylenir a...,mecaz,Üstüne almak,Zero-shot
3,Arkadaş gruplarında sıkça karşılaşılan sorunla...,mecaz,Üstüne almak,Zero-shot
4,Herkes projedeki başarısızlıkla ilgili endişel...,mecaz,Üstüne almak,Zero-shot
5,Grup içinde yapılan uyarıları kimse ciddiye al...,mecaz,Üstüne almak,Zero-shot
6,Toplantıda dile getirilen eksiklikler üzerine ...,mecaz,Üstüne almak,Zero-shot
7,"Kimse kabul etmek istemezken, sorunun farkında...",mecaz,Üstüne almak,Zero-shot
8,Her ne kadar kimsenin hatası olarak görünmese ...,mecaz,Üstüne almak,Zero-shot
9,Ekip üyeleri genellikle hataları kabul etmekte...,mecaz,Üstüne almak,Zero-shot


In [890]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,Toplantıdaki eleştirileri üstüne alındı ve hem...,mecaz,Üstüne almak,Zero-shot
1,"Hoca sınıfta genel bir uyarı yaptığında, herke...",mecaz,Üstüne almak,Zero-shot
2,Projedeki aksaklıklar geçtiğimiz hafta ortaya ...,mecaz,Üstüne almak,Zero-shot
3,Her ne kadar takım olarak kaybetmiş olsalar da...,mecaz,Üstüne almak,Zero-shot
4,Öğretmen ders anlatırken yapılan hataları öğre...,mecaz,Üstüne almak,Zero-shot
5,Öğretmen sınıfta disiplinsizlik üzerine konuşu...,mecaz,Üstüne almak,Zero-shot
6,Yönetici eleştirileri üstüne alarak tüm soruml...,mecaz,Üstüne almak,Zero-shot
7,Ahmet dedikoduları üstüne alındı ve huzursuz o...,mecaz,Üstüne almak,Zero-shot
8,"Müdür genel bir uyarı yaptığında, bütün ekip s...",mecaz,Üstüne almak,Zero-shot
9,Sunum sırasında yapılan eleştirilere kimse tep...,mecaz,Üstüne almak,Zero-shot


**Enhanced-prompting**

In [891]:
gerçek_örnekler=['"Ağzı yanmak" deyimi sıcak bir içecekten, sıcak bir yiyecekten, sıcak buhardan, sıcak yüzeylere dokunmaktan, acı bir yemekten ya da kimyasal maddelere maruz kalmaktan kaynaklanan durumlar için kullanılabilir.',
                 '"Burnunu sürtmek" deyimi sert bir zemine düşmekten, bir duvara ya da sert bir yüzeye çarpmaktan, sert bir zeminde yuvarlanmaktan, bir nesneye sürtünmekten ya da yere düşerken yüzüne darbe almaktan kaynaklanan durumlar için kullanılabilir.',
                 '"Ateşle oynamak" deyimi bir ateş kaynağına dokunmaktan, yanıcı maddelerle deney yapmaktan, açık alevlerle çalışmaktan, ateşle dolu bir alanda bulunmaktan ya da ateşle tehlikeli bir şekilde oynamaktan kaynaklanan durumlar için kullanılabilir.',
                 '"Kuyruğuna basmak" deyimi bir hayvanın kuyruğunun yanlışlıkla basılmasından, bir hayvanın kuyruğunu kapıya sıkıştırmaktan, bir hayvanın kuyruğunu bir nesneyle vurmak ya da üzerine düşmekten, ya da bir hayvanın kuyruğunu çekmekten kaynaklanan durumlar için kullanılabilir.',
                 '"Çamur atmak" deyimi bir kişinin fiziksel olarak çamur fırlatmasından, bir grup insan arasında çamur topları oynarken birbirlerine çamur atılmasından, bir çocuğun yağmurlu bir günde çamurlu bir alanda oynarken çamur sıçratmasından, bir inşaat alanında veya bir tarlada çalışırken çamurlu ellerle birbirine dokunmaktan ya da bir festival veya kutlamada çamur banyosu yaparken çamur atılmasından kaynaklanan durumlar için kullanılabilir.']

In [892]:
mecaz_örnekler = ['"Ağzı yanmak" deyimi birine güvenip aldatıldığında, ekonomik bir zarar yaşandığında, kişisel bir hüsranla karşılaşıldığında, yanıltıcı bir bilgiye dayanarak yanılgıya düştüğünde ya da beklenen bir sonucun tam tersiyle karşılaşıldığında kullanılabilir'
                  '"Ateşle oynamak" deyimi mecaz anlamda, tehlikeli bir kişiyle işbirliği yaparken, riskli bir finansal yatırım yaparken, yasal sınırları zorlayacak bir eyleme girişirken, toplumsal normlara karşı gelindiğinde ya da genel kabul görmeyen bir stratejiyi benimserken kullanılabilir.',
                  '"Kuyruğuna basmak" deyimi bir kişinin hassas bir konusuna dokunduğunda, başkasının gizli zayıflıklarını ortaya çıkardığında, birisinin özel alanına izinsiz girildiğinde, birinin geçmiş hatalarını gündeme getirdiğinde ya da bir kişinin gizli planlarını bozduğunda kullanılabilir.',
                  '"Çamur atmak" deyimi bir kişiye iftira atıldığında, bir rakibin itibarını zedelemek için yanlış bilgiler yayıldığında, bir kişi hakkında asılsız dedikodular çıkarıldığında, birisinin kamu önündeki imajına zarar vermek amacıyla negatif yorumlar yapıldığında ya da bir kişinin başarılarını gölgelemek için kasıtlı olarak kötüleyici söylemlerde bulunulduğunda kullanılabilir.',
                  '"Burnunu sürtmek" deyimi bir hata yaptıktan sonra alay konusu olduğunda, başarısız bir girişimden sonra eleştiri aldığında, bir konuda beklenmedik bir yenilgi yaşandığında, hatalı bir kararın sonuçlarına katlanıldığında ya da bir kişinin gururu kırıldığında kullanılabilir.']

In [893]:
type = "Enhanced-prompting"
step = 1

In [894]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. İstenilen isteğe göre yanıt vermeye özen göster. Deyimin kelime köklerini değiştirmeyerek. İnsan isimlerini kullanarak cümle kurmamalısın !"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

if category == "gerçek":
    örnek = random.choice(gerçek_örnekler)
    prompt = f"""
    {örnek} Benzer bir şekilde, "{idiom}" deyiminin {category} anlamda kullanıldığı durumları listeler misin?
    """
else:
    örnek = random.choice(mecaz_örnekler)
    prompt = f"""
    {örnek} Benzer bir şekilde, "{idiom}" deyiminin {category} anlamda kullanıldığı durumları listeler misin?
    """


In [895]:
chat_session = ChatSession(category=category)

Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. İstenilen isteğe göre yanıt vermeye özen göster. Deyimin kelime köklerini değiştirmeyerek. İnsan isimlerini kullanarak cümle kurmamalısın !


In [896]:
print(prompt)
print("\n")

for i in range(step):
    result = chat_session.get_completion(prompt)
    print(result)


    "Ağzı yanmak" deyimi birine güvenip aldatıldığında, ekonomik bir zarar yaşandığında, kişisel bir hüsranla karşılaşıldığında, yanıltıcı bir bilgiye dayanarak yanılgıya düştüğünde ya da beklenen bir sonucun tam tersiyle karşılaşıldığında kullanılabilir"Ateşle oynamak" deyimi mecaz anlamda, tehlikeli bir kişiyle işbirliği yaparken, riskli bir finansal yatırım yaparken, yasal sınırları zorlayacak bir eyleme girişirken, toplumsal normlara karşı gelindiğinde ya da genel kabul görmeyen bir stratejiyi benimserken kullanılabilir. Benzer bir şekilde, "Üstüne almak" deyiminin mecaz anlamda kullanıldığı durumları listeler misin?
    




Elbette, "üstüne almak" deyimi genellikle bir sorumluluğu kabul etmek, suçlamayı veya eleştiriyi kabullenmek veya bir görevi üzerine almak anlamında kullanılır. Bu deyim, çeşitli bağlamlarda mecaz anlamda ifade edilebilir:

1. Bir projenin ya da işin sorumluluğunu üzerine almak: Bu durum, bir kişinin özellikle zor veya büyük bir işte liderlik etmeyi kabul etmesiyle ilgilidir.
2. Eleştiri veya suçlamayı kabullenmek: Bir hata veya problem ortaya çıktığında, kişinin suçu veya hataları üzerine alarak diğerlerinin üzerinden bu yükü kaldırması söz konusu olabilir.
3. Bir konuda inisiyatif kullanmak: Belirli bir konuda gereken adımları atan ve o konuda girişimde bulunan kişi için kullanılır.
4. Bir durumun veya problemle ilgilenmeyi kabul etmek: Bir meseleyle veya zorluğu doğrudan ele almayı üstlenmek anlamında kullanılabilir.
5. Acil durumda liderlik yapmak: Beklenmedik bir durumda ön planda olmayı ve yönetimi ele almayı ifade eder.


In [897]:
prompt = f"""
"{idiom}" deyiminin bir önceki mesajda listelenen her bir durum için {category} anlamda yer aldığı dört farklı cümle oluştur. İlk cümle uzun ve detaylı, ikinci cümle soru formunda, üçüncü cümle olumsuz yapıda, dördüncü cümlede deyimi oluşturan kelimeler arasına başka kelimeler girmiş biçimde olmalı. Lütfen bu deyimin kelime kökleri değiştirmeden. Her kategori için 4 farklı cümle oluşturmalısın.Yanıt olarak sadace bu cümleleri dön. Deyimin kelime köklerini değiştirmeyerek.
"""

In [898]:
print(prompt)
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


"Üstüne almak" deyiminin bir önceki mesajda listelenen her bir durum için mecaz anlamda yer aldığı dört farklı cümle oluştur. İlk cümle uzun ve detaylı, ikinci cümle soru formunda, üçüncü cümle olumsuz yapıda, dördüncü cümlede deyimi oluşturan kelimeler arasına başka kelimeler girmiş biçimde olmalı. Lütfen bu deyimin kelime kökleri değiştirmeden. Her kategori için 4 farklı cümle oluşturmalısın.Yanıt olarak sadace bu cümleleri dön. Deyimin kelime köklerini değiştirmeyerek.



In [899]:
responses

['1. Bir projenin ya da işin sorumluluğunu üzerine almak:\n   - Projeyi başarıya ulaştırmak için gereken her türlü çabayı göstermeye hazır olduğunu belirten baş mühendis, tüm zorlukları göz önünde bulundurarak bu büyük sorumluluğu üstüne aldı.\n   - Bu zorlu projenin sorumluluğunu üstüne almak istiyor musun?\n   - Ne yazık ki, kimsenin bu karmaşık projenin sorumluluğunu üstüne almak istemediği görülüyor.\n   - Tüm zorluklarına rağmen, bu projenin sorumluluğunu tune bir biçimde üzerine alarak herkesi şaşırttı.\n\n2. Eleştiri veya suçlamayı kabullenmek:\n   - Tartışmanın ardından, söylediği sözlerin yarattığı yanlış anlamaları düzeltmek adına tüm suçlamaları üstüne alarak hatalarını kabul etti.\n   - Yapılan bu hatalar için üstüne almak gerektiğinin farkında mısın?\n   - Kimse açıkça hatasını kabul edip eleştiriyi üstüne almak istemedi.\n   - Verilen yoğun eleştirilere rağmen, hatalarını bir üstüne alarak kabullenmekten çekinmedi.\n\n3. Bir konuda inisiyatif kullanmak:\n   - Pazarlama de

In [244]:
data = [
    {"submission": "Uzun bir günün ardından yatağına uzanan ve gözlerini yavaşça yumarak, kendini uykunun kollarına bırakan Ahmet, tüm stres ve yorgunluklarını bu şekilde bir kenara atıyor.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Sen de gece yatarken gözlerini yumar mısın genellikle?", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Uykuya dalmadan önce gözlerimi asla hemen yummam, genellikle bir süre tavana bakarım.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Uykuya dalmadan önce, genellikle gözlerimi yavaş yavaş yumarak, rahatlamayı tercih ederim.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Yaz günlerinde parkta koşarken aniden karşısına çıkan güneşin parlak ışığına doğrudan maruz kalan Elif, refleks olarak gözlerini yumar ve başını biraz öne eğer.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Güneşe bakarken gözlerini yumar mısın, yoksa başka bir yöntem kullanır mısın?", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Güneşe doğrudan bakarken, gözlerimi yummam asla iyi bir fikir değil, çünkü bu, görsel algılamamı tamamen engelliyor.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Güneşin kuvvetli ışığına maruz kaldığında, çoğu insan doğal olarak gözlerini bir anlığına yumarak korumaya alır.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Göz doktoruna ziyarette bulunan Murat, görme keskinliğini değerlendirmek için yapılan test sırasında, doktorun talimatları üzerine sol gözünü yumar, sağ gözüyle harfleri okumaya çalışır.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Göz sağlığı kontrolünde doktorun isteği üzerine bir gözünü yumar mıydın?", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Göz doktoru bana bir gözümü yumarak harfleri okumamı söylemişti, fakat ben yanlışlıkla iki gözümü de yummadım.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Göz muayenesi sırasında, genellikle doktorun yönlendirmesi ile bir gözümü kısa süreliğine yumarak diğer gözümün görme yeteneğini test ederim.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Düğünde fotoğrafçı, fotoğrafını çekmek üzere olduğu sırada aniden flaşı patlatınca, Deniz doğal bir reaksiyon olarak hızla gözlerini yumar ve birkaç saniye sonra yeniden açar.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Fotoğraf çekerken birdenbire flaş patladığında gözlerini yumar mısın?", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Fotoğraf çekilirken genelde gözlerimi yummam, çünkü flaşın gözüme zarar vereceğini düşünmüyorum.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},
    {"submission": "Flaşın ani patlamasıyla genellikle hemen gözlerimi bir refleks olarak yumarak kendimi koruma altına alırım.", "category": "gerçek", "idiom": "Göz yummak", "type": "Enhanced-prompting"},

]
new_df=pd.DataFrame(data)
new_df

,submission,category,idiom,type
0,Uzun bir günün ardından yatağına uzanan ve göz...,gerçek,Göz yummak,Enhanced-prompting
1,Sen de gece yatarken gözlerini yumar mısın gen...,gerçek,Göz yummak,Enhanced-prompting
2,Uykuya dalmadan önce gözlerimi asla hemen yumm...,gerçek,Göz yummak,Enhanced-prompting
3,"Uykuya dalmadan önce, genellikle gözlerimi yav...",gerçek,Göz yummak,Enhanced-prompting
4,Yaz günlerinde parkta koşarken aniden karşısın...,gerçek,Göz yummak,Enhanced-prompting
5,"Güneşe bakarken gözlerini yumar mısın, yoksa b...",gerçek,Göz yummak,Enhanced-prompting
6,"Güneşe doğrudan bakarken, gözlerimi yummam asl...",gerçek,Göz yummak,Enhanced-prompting
7,"Güneşin kuvvetli ışığına maruz kaldığında, çoğ...",gerçek,Göz yummak,Enhanced-prompting
8,"Göz doktoruna ziyarette bulunan Murat, görme k...",gerçek,Göz yummak,Enhanced-prompting
9,Göz sağlığı kontrolünde doktorun isteği üzerin...,gerçek,Göz yummak,Enhanced-prompting


In [900]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual lines
        lines = block.split('\n')
        # Skip the header line (assumes the header is the first line)
        for line in lines[1:]:  # this skips the header which is lines[0]
            # Check if the line contains an actual sentence
            if line.strip() and '-' in line:
                # Remove the bullet point and any leading/trailing spaces
                sentence = line.split('-', 1)[-1].strip()
                # Append the structured tuple
                structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [901]:
def parse_and_extract_sentences(raw_responses_list):
    # Initialize the list to store sentences
    extracted_sentences = []
    
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split the block into lines
        lines = block.split('\n')
        
        # Iterate through each line in the block
        for line in lines:
            # Strip whitespace from line
            line = line.strip()
            
            # Check if line starts with a dash followed by a space, indicating a sentence
            if line.startswith('. '):
                # Extract the sentence after the dash and strip leading/trailing whitespace
                sentence = line[2:].strip()
                # Append the cleaned sentence to the list
                extracted_sentences.append((sentence, category, idiom, type))
    
    return extracted_sentences

In [902]:
def parse_numbered_sentences(raw_texts):
    # Initialize the list to store extracted sentences
    extracted_sentences = []
    
    # Iterate through each text block in the list
    for raw_text in raw_texts:
        # Split the input text into lines
        lines = raw_text.split('\n')
        
        # Iterate through each line in the input text
        for line in lines:
            # Strip any leading/trailing whitespace from the line
            line = line.strip()
            
            # Check if the line starts with a numeric prefix followed by a dot and space (e.g., '1. ')
            if line.startswith(('1. ', '2. ', '3. ', '4. ')):
                # Extract the sentence after the numeric prefix
                sentence = line.split('. ', 1)[-1]
                # Append the extracted sentence to the list
                extracted_sentences.append((sentence, category, idiom, type))
    
    return extracted_sentences

In [903]:
parsed_data = parse_and_structure_data(responses, idiom, category, type)
parsed_data

[('Projeyi başarıya ulaştırmak için gereken her türlü çabayı göstermeye hazır olduğunu belirten baş mühendis, tüm zorlukları göz önünde bulundurarak bu büyük sorumluluğu üstüne aldı.',
  'mecaz',
  'Üstüne almak',
  'Enhanced-prompting'),
 ('Bu zorlu projenin sorumluluğunu üstüne almak istiyor musun?',
  'mecaz',
  'Üstüne almak',
  'Enhanced-prompting'),
 ('Ne yazık ki, kimsenin bu karmaşık projenin sorumluluğunu üstüne almak istemediği görülüyor.',
  'mecaz',
  'Üstüne almak',
  'Enhanced-prompting'),
 ('Tüm zorluklarına rağmen, bu projenin sorumluluğunu tune bir biçimde üzerine alarak herkesi şaşırttı.',
  'mecaz',
  'Üstüne almak',
  'Enhanced-prompting'),
 ('Tartışmanın ardından, söylediği sözlerin yarattığı yanlış anlamaları düzeltmek adına tüm suçlamaları üstüne alarak hatalarını kabul etti.',
  'mecaz',
  'Üstüne almak',
  'Enhanced-prompting'),
 ('Yapılan bu hatalar için üstüne almak gerektiğinin farkında mısın?',
  'mecaz',
  'Üstüne almak',
  'Enhanced-prompting'),
 ('Kimse 

In [904]:
new_df= pd.DataFrame(parsed_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,Projeyi başarıya ulaştırmak için gereken her t...,mecaz,Üstüne almak,Enhanced-prompting
1,Bu zorlu projenin sorumluluğunu üstüne almak i...,mecaz,Üstüne almak,Enhanced-prompting
2,"Ne yazık ki, kimsenin bu karmaşık projenin sor...",mecaz,Üstüne almak,Enhanced-prompting
3,"Tüm zorluklarına rağmen, bu projenin sorumlulu...",mecaz,Üstüne almak,Enhanced-prompting
4,"Tartışmanın ardından, söylediği sözlerin yarat...",mecaz,Üstüne almak,Enhanced-prompting
5,Yapılan bu hatalar için üstüne almak gerektiği...,mecaz,Üstüne almak,Enhanced-prompting
6,Kimse açıkça hatasını kabul edip eleştiriyi üs...,mecaz,Üstüne almak,Enhanced-prompting
7,"Verilen yoğun eleştirilere rağmen, hatalarını ...",mecaz,Üstüne almak,Enhanced-prompting
8,Pazarlama departmanında yaşanan düşüşü analiz ...,mecaz,Üstüne almak,Enhanced-prompting
9,Bu krizi yönetmek için inisiyatifini üstüne al...,mecaz,Üstüne almak,Enhanced-prompting


In [905]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,Toplantıdaki eleştirileri üstüne alındı ve hem...,mecaz,Üstüne almak,Zero-shot
1,"Hoca sınıfta genel bir uyarı yaptığında, herke...",mecaz,Üstüne almak,Zero-shot
2,Projedeki aksaklıklar geçtiğimiz hafta ortaya ...,mecaz,Üstüne almak,Zero-shot
3,Her ne kadar takım olarak kaybetmiş olsalar da...,mecaz,Üstüne almak,Zero-shot
4,Öğretmen ders anlatırken yapılan hataları öğre...,mecaz,Üstüne almak,Zero-shot
...,...,...,...,...
71,"Zor zamanlarda, liderler genellikle inisiyatif...",mecaz,Üstüne almak,Enhanced-prompting
72,Müşteri şikayetlerini daha etkin bir şekilde ç...,mecaz,Üstüne almak,Enhanced-prompting
73,Bu büyük sorunla ilgilenmek için gönüllü olara...,mecaz,Üstüne almak,Enhanced-prompting
74,Çevresel düzenlemeler konusunda hiç kimse en k...,mecaz,Üstüne almak,Enhanced-prompting


In [2332]:
prompt = f"""
Diğer kategoriler ile devam eder misin aynı şekilde ?
"""

In [2333]:
print(prompt)
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


Diğer kategoriler ile devam eder misin aynı şekilde ?



In [2334]:
responses

["**Eğitimde Reformlar Yapmak:**\n1. Maria Montessori, ona adını veren Montessori eğitim sistemiyle, çocukların özgürce keşfetmelerine ve kendi hızlarında öğrenmelerine olanak tanıyarak eğitim dünyasında devrim niteliğinde bir iz bıraktı.\n2. Dijital eğitim araçları kullanılarak eğitimde daha kalıcı bir iz bırakmak mümkün müdür?\n3. Birçok yeni eğitim modeli, sadece kısa süreli ilgi çekiyor ve uzun vadeli bir iz bırakamıyor.\n4. Finlandiya, öğrenci merkezli yaklaşımı ve yenilikçi eğitim politikaları ile uluslararası alanda başarı grafiğinin sürekli yükselmesiyle eğitim dünyasında sağlam bir iz bıraktı.\n \n**İş Dünyasında Yenilikçilik Yapmak:**\n1. Steve Jobs'un Apple şirketindeki vizyoner liderliği, teknoloji ve tasarımın bir araya geldiği ürünler yaratılmasını sağlayarak tüketici elektroniği sektöründe çağ atlatan bir iz bıraktı.\n2. Sürdürülebilirlik bazlı iş modeli geliştirerek iş dünyasında gerçekten bir iz bırakılabilir mi?\n3. Pek çok startup, piyasada sıra dışı bir etki yaratma

In [2335]:
parsed_data = parse_numbered_sentences(responses)
parsed_data

[('Maria Montessori, ona adını veren Montessori eğitim sistemiyle, çocukların özgürce keşfetmelerine ve kendi hızlarında öğrenmelerine olanak tanıyarak eğitim dünyasında devrim niteliğinde bir iz bıraktı.',
  'mecaz',
  'İz bırakmak',
  'Enhanced-prompting'),
 ('Dijital eğitim araçları kullanılarak eğitimde daha kalıcı bir iz bırakmak mümkün müdür?',
  'mecaz',
  'İz bırakmak',
  'Enhanced-prompting'),
 ('Birçok yeni eğitim modeli, sadece kısa süreli ilgi çekiyor ve uzun vadeli bir iz bırakamıyor.',
  'mecaz',
  'İz bırakmak',
  'Enhanced-prompting'),
 ('Finlandiya, öğrenci merkezli yaklaşımı ve yenilikçi eğitim politikaları ile uluslararası alanda başarı grafiğinin sürekli yükselmesiyle eğitim dünyasında sağlam bir iz bıraktı.',
  'mecaz',
  'İz bırakmak',
  'Enhanced-prompting'),
 ("Steve Jobs'un Apple şirketindeki vizyoner liderliği, teknoloji ve tasarımın bir araya geldiği ürünler yaratılmasını sağlayarak tüketici elektroniği sektöründe çağ atlatan bir iz bıraktı.",
  'mecaz',
  'İ

In [2336]:
new_df= pd.DataFrame(parsed_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,"Maria Montessori, ona adını veren Montessori e...",mecaz,İz bırakmak,Enhanced-prompting
1,Dijital eğitim araçları kullanılarak eğitimde ...,mecaz,İz bırakmak,Enhanced-prompting
2,"Birçok yeni eğitim modeli, sadece kısa süreli ...",mecaz,İz bırakmak,Enhanced-prompting
3,"Finlandiya, öğrenci merkezli yaklaşımı ve yeni...",mecaz,İz bırakmak,Enhanced-prompting
4,Steve Jobs'un Apple şirketindeki vizyoner lide...,mecaz,İz bırakmak,Enhanced-prompting
5,Sürdürülebilirlik bazlı iş modeli geliştirerek...,mecaz,İz bırakmak,Enhanced-prompting
6,"Pek çok startup, piyasada sıra dışı bir etki y...",mecaz,İz bırakmak,Enhanced-prompting
7,"Elon Musk, risk almayı seven liderliği ve yeni...",mecaz,İz bırakmak,Enhanced-prompting
8,"J.R.R. Tolkien, Orta Dünya'yı yaratmak ve bu k...",mecaz,İz bırakmak,Enhanced-prompting
9,Yeni medya araçlarını kullanarak kültürde kalı...,mecaz,İz bırakmak,Enhanced-prompting


In [2337]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,"Büyük yazar, kalemiyle edebiyat dünyasında der...",mecaz,İz bırakmak,Zero-shot
1,"O başarılı bilim insanı, tıp alanında yaptığı ...",mecaz,İz bırakmak,Zero-shot
2,"Hayırsever iş insanı, yaptığı yardımlarla topl...",mecaz,İz bırakmak,Zero-shot
3,"Olağanüstü bir lider olan o, halkının kalbinde...",mecaz,İz bırakmak,Zero-shot
4,"Yenilikçi fikirleriyle genç girişimci, teknolo...",mecaz,İz bırakmak,Zero-shot
...,...,...,...,...
83,"Elon Musk, risk almayı seven liderliği ve yeni...",mecaz,İz bırakmak,Enhanced-prompting
84,"J.R.R. Tolkien, Orta Dünya'yı yaratmak ve bu k...",mecaz,İz bırakmak,Enhanced-prompting
85,Yeni medya araçlarını kullanarak kültürde kalı...,mecaz,İz bırakmak,Enhanced-prompting
86,"Birçok çağdaş sanatçı, anlamlı kültürel diyalo...",mecaz,İz bırakmak,Enhanced-prompting


In [906]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. Lütfen cümlelerinizi tekrar kullanmaktan kaçının ve deyimleri doğru ve {category} anlamıyla kullanarak cümleler oluşturun. Deyimin kelime köklerini değiştirmeyerek. İnsan isimlerini kullanarak cümle kurmamalısın !"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [907]:
chat_session = ChatSession(category=category)

Türkçe, anadiliniz olduğundan, dilimizi zengin ve yaratıcı biçimlerde kullanmalısın. Lütfen cümlelerinizi tekrar kullanmaktan kaçının ve deyimleri doğru ve mecaz anlamıyla kullanarak cümleler oluşturun. Deyimin kelime köklerini değiştirmeyerek. İnsan isimlerini kullanarak cümle kurmamalısın !


In [908]:
prompt = f"""
"{idiom}" deyimini {category} anlamda kullanarak anlamda kullanarak, farklı cümle yapılarıyla 6 yeni cümle oluştur. Deyimi cümlenin farklı bölümlerinde kullan.Lütfen cümlelerinizi tekrar kullanmaktan kaçının
"""
print(prompt)
print("\n")
responses = []
for i in range(4):
    result = chat_session.get_completion(prompt)
    responses.append(result)


"Üstüne almak" deyimini mecaz anlamda kullanarak anlamda kullanarak, farklı cümle yapılarıyla 6 yeni cümle oluştur. Deyimi cümlenin farklı bölümlerinde kullan.Lütfen cümlelerinizi tekrar kullanmaktan kaçının





In [909]:
responses

['1. Kimse suçu kabul etmek istemezken, o, tüm hataları üstüne alarak cesur bir tavır sergiledi.\n2. Gerçekten hatalı olanın kim olduğunu bilmemesine rağmen, arkadaşının üstüne alındığı anlaşmazlıkta haksız yere eleştirildi.\n3. Herkesin sessiz kaldığı toplantıda, sorumluluğu üstüne almak onun liderlik niteliklerini gözler önüne serdi.\n4. Projede yaşanan aksaklıklar konuşulurken, olayı direkt üstüne alarak tüm ekip adına özür diledi.\n5. İş yerinde yaşanan sorunları her zaman üstüne alan kişi, çözüm yollarını da en hızlı bulandır.\n6. Kriz anında gereken adımları atan ve sorunun çözümünü üstüne almakta tereddüt etmeyen yönetici, tüm çalışanlar tarafından takdir topladı.',
 '1. Eleştirileri üstüne alarak iyileştirme yolunda önemli bir adım attı.\n2. Fikri beğenilmediğinde, başarısızlığı üstüne alıp, daha iyisini yapmak için kolları sıvadı.\n3. Herkesin çekindiği projeyi üstüne alarak kendini kanıtlama fırsatı buldu.\n4. Yanlış anlaşılan bir durumda, suçu üstüne alarak gergin havayı yum

In [910]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [911]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,"Kimse suçu kabul etmek istemezken, o, tüm hata...",mecaz,Üstüne almak,Enhanced-prompting
1,Gerçekten hatalı olanın kim olduğunu bilmemesi...,mecaz,Üstüne almak,Enhanced-prompting
2,"Herkesin sessiz kaldığı toplantıda, sorumluluğ...",mecaz,Üstüne almak,Enhanced-prompting
3,"Projede yaşanan aksaklıklar konuşulurken, olay...",mecaz,Üstüne almak,Enhanced-prompting
4,İş yerinde yaşanan sorunları her zaman üstüne ...,mecaz,Üstüne almak,Enhanced-prompting
5,Kriz anında gereken adımları atan ve sorunun ç...,mecaz,Üstüne almak,Enhanced-prompting
6,Eleştirileri üstüne alarak iyileştirme yolunda...,mecaz,Üstüne almak,Enhanced-prompting
7,"Fikri beğenilmediğinde, başarısızlığı üstüne a...",mecaz,Üstüne almak,Enhanced-prompting
8,Herkesin çekindiği projeyi üstüne alarak kendi...,mecaz,Üstüne almak,Enhanced-prompting
9,"Yanlış anlaşılan bir durumda, suçu üstüne alar...",mecaz,Üstüne almak,Enhanced-prompting


In [912]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,Toplantıdaki eleştirileri üstüne alındı ve hem...,mecaz,Üstüne almak,Zero-shot
1,"Hoca sınıfta genel bir uyarı yaptığında, herke...",mecaz,Üstüne almak,Zero-shot
2,Projedeki aksaklıklar geçtiğimiz hafta ortaya ...,mecaz,Üstüne almak,Zero-shot
3,Her ne kadar takım olarak kaybetmiş olsalar da...,mecaz,Üstüne almak,Zero-shot
4,Öğretmen ders anlatırken yapılan hataları öğre...,mecaz,Üstüne almak,Zero-shot
...,...,...,...,...
95,"Proje başarısız olduğunda, yaşananların soruml...",mecaz,Üstüne almak,Enhanced-prompting
96,"Eleştirileri üstüne almak yerine, yapıcı geri ...",mecaz,Üstüne almak,Enhanced-prompting
97,"Herkes hatayı kabul etmekten kaçınırken, sorun...",mecaz,Üstüne almak,Enhanced-prompting
98,İş yerindeki her türlü aksaklığı üstüne alarak...,mecaz,Üstüne almak,Enhanced-prompting


In [913]:
# Define the path for the CSV file
file_path = 'gpt4_tr_mecaz.csv'

# Check if the file exists
if os.path.exists(file_path):
    # Append without writing headers
    df_structured.to_csv(file_path, mode='a', index=False, header=False)
else:
    # Create a new file and write the DataFrame with headers
    df_structured.to_csv(file_path, mode='w', index=False, header=True)